# Self-Driving Car Engineer Nanodegree

## Project: **Finding Lane Lines on the Road** 

## Import Packages

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
%matplotlib inline

In [ ]:
import math

class mean_tracker(object):
    # helper static class to keep track of mean slope/intercept values
    # call reset method before processing new series of images
    mean_over_last_n = 5
    slopes = [np.array([]), np.array([])]
    intercepts = [np.array([]), np.array([])]
    
    @staticmethod
    def reset(n = 5):
        mean_tracker.mean_over_last_n = n
        mean_tracker.slopes = [np.array([]), np.array([])]
        mean_tracker.intercepts = [np.array([]), np.array([])]
        
    @staticmethod
    def add_and_get(slope, intercept):
        # keep left and right lane marks in the separate arrays
        idx = 0 if slope < 0 else 1
        if (mean_tracker.slopes[idx].size >= mean_tracker.mean_over_last_n):
            mean_tracker.slopes[idx] = np.append(mean_tracker.slopes[idx][-mean_tracker.mean_over_last_n+1:], slope)
            mean_tracker.intercepts[idx] = np.append(mean_tracker.intercepts[idx][-mean_tracker.mean_over_last_n+1:], intercept)
            slope = np.mean(mean_tracker.slopes[idx])
            intercept = np.mean(mean_tracker.intercepts[idx])
        else:
            mean_tracker.slopes[idx] = np.array([slope for i in range(0, mean_tracker.mean_over_last_n)])
            mean_tracker.intercepts[idx] = np.array([intercept for i in range(0, mean_tracker.mean_over_last_n)])             
        return [slope, intercept]

def mask_lanes(image):
    # Apply a mask to keep only "white" and "yellow" colors
    lower_yellow = np.array([128,128,0])
    upper_yellow = np.array([255,255,100])
    yellow_mask = cv2.inRange(image, lower_yellow, upper_yellow)
    
    lower_white = np.array([220,220,220])
    upper_white = np.array([255,255,255])
    white_mask = cv2.inRange(image, lower_white, upper_white)
    
    mask = cv2.add(white_mask, yellow_mask)
    blurred_mask = cv2.GaussianBlur(mask,(5, 5),0)
    
    return cv2.bitwise_and(image,image, mask= blurred_mask)

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    # Applies the Canny transform
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    # Applies a Gaussian Noise kernel
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    # defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    # filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    # returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    # `img` should be the output of a Canny transform.
    return cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

def lane_line_filter(img_center):
    # treat lines with with unexpected slopes as noise
    def noise_filter(line):
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1)
            return (slope < -0.4 and x1 < img_center) or (slope > 0.4 and x1 > img_center)
    return noise_filter

def filter_lines(lines, img_center):
    return list(filter(lane_line_filter(img_center), lines))

def is_left_line(line, img_center):
    for x1,y1,x2,y2 in line:
        return x1 < img_center

def partition_lines(lines, img_center):
    # partition lines to the left/right by horizontal location relative to the center of the image
    left_lines, right_lines = [], []
    for line in lines:
        (left_lines, right_lines)[is_left_line(line, img_center)].append(line)
    return left_lines, right_lines
    
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def extract_xy(lines):
    xs, ys = [], []
    for line in lines:
        for x1,y1,x2,y2 in line:
            xs.extend([x1, x2])
            ys.extend([y1, y2])
    return xs, ys

def fit_line(lines, low_y, high_y):
    # aproximates lines with a single line using the running mean value
    [xs, ys] = extract_xy(lines);
    [slope, intercept] = np.polyfit(xs, ys, 1)
    [slope, intercept] = mean_tracker.add_and_get(slope, intercept)
    
    low_x = (low_y - intercept) / slope
    high_x = (high_y - intercept) / slope
    
    return [[int(low_x), int(low_y), int(high_x), int(high_y)]]

def draw_raw_lines(lines, img):
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, [255, 0, 0], 4)
    return line_img

def draw_mean_lines(lines, img):
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    low_y = 0.65 * img.shape[0]
    high_y = img.shape[0]
    img_center = img.shape[1] / 2
    
    # categorize as left/right
    [left_lines, right_lines] = partition_lines(lines, img_center)
    draw_lines(line_img, [fit_line(left_lines, low_y, high_y)], [255, 0, 0], 14)
    draw_lines(line_img, [fit_line(right_lines, low_y, high_y)], [0, 0, 255], 14)
    
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(initial_img, img, α=0.8, β=1., λ=0.):    
    # The result image is computed as follows: initial_img * α + img * β + λ    
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Lane Finding Pipeline

In [ ]:
gauss_kernel = 7

canny_low_threshold = 50
canny_high_threshold = 150

rho = 1 # distance resolution in pixels of the Hough grid
theta = np.pi/180 # angular resolution in radians of the Hough grid
threshold = 15     # minimum number of votes (intersections in Hough grid cell)
min_line_length = 3 # minimum number of pixels making up a line
max_line_gap = 5   # maximum gap in pixels between connectable line segments

def region_of_interest_points(xsize, ysize):
    left_bottom = [0, ysize]
    right_bottom = [xsize, ysize]
    left_top = [xsize* 0.45, ysize * 0.60]
    right_top = [xsize* 0.55, ysize * 0.60]
    return np.array([[left_bottom, right_bottom, right_top, left_top]], np.int32)    

def image_pipeline(image, reset_mean = False):
    if reset_mean:
        mean_tracker.reset()
    
    ysize = image.shape[0]
    xsize = image.shape[1]
    img_center = image.shape[1] / 2
    
    pts_of_interest = region_of_interest_points(xsize, ysize)
    
    masked_lanes = mask_lanes(image)
    
    gray = grayscale(masked_lanes)    
    blured = gaussian_blur(gray, gauss_kernel)
            
    edges = canny(blured, canny_low_threshold, canny_high_threshold)
            
    region = region_of_interest(edges, [pts_of_interest])
    
    h_lines = hough_lines(region, rho, theta, threshold, min_line_length, max_line_gap)
   
    filtered_lines = filter_lines(h_lines, img_center)
    
    lines_img = draw_mean_lines(filtered_lines, image)
    
    return weighted_img(image, lines_img)

## Test on Images

In [ ]:
def show_image(img, cmap=None, save_as=None):
    my_dpi=96
    height=540
    width=960
    
    fig = plt.figure(figsize=(width/my_dpi, height/my_dpi), dpi=my_dpi, frameon=False)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')
    plt.imshow(img, cmap)
    
    if save_as != None:
        plt.savefig(save_as)

# testing individual functions of the pipeline
mean_tracker.reset()

pipeline_demo_output_dir = 'pipeline_demo/'
image = mpimg.imread('test_images/solidYellowLeft-0291_pre.jpg')
show_image(image)

ysize = image.shape[0]
xsize = image.shape[1]
img_center = image.shape[1] / 2

pts_of_interest = region_of_interest_points(xsize, ysize)

masked_lanes = mask_lanes(image)
show_image(masked_lanes, None, pipeline_demo_output_dir + '1_masked_lanes.jpg')

gray = grayscale(masked_lanes)
show_image(gray, 'gray', pipeline_demo_output_dir + '2_gray.jpg')

blured = gaussian_blur(gray, gauss_kernel)
show_image(blured, 'gray', pipeline_demo_output_dir + '3_blured.jpg')

edges = canny(blured, canny_low_threshold, canny_high_threshold)
show_image(edges, 'gray', pipeline_demo_output_dir + '4_edges.jpg')

region = region_of_interest(edges, [pts_of_interest])
show_image(region, 'gray', pipeline_demo_output_dir + '5_region.jpg')

h_lines = hough_lines(region, rho, theta, threshold, min_line_length, max_line_gap)
raw_lines_img = draw_raw_lines(h_lines, image)
show_image(weighted_img(image, raw_lines_img), None, pipeline_demo_output_dir + '6_raw_lines.jpg')

filtered_lines = filter_lines(h_lines, img_center)
raw_filtered_lines_img = draw_raw_lines(filtered_lines, image)
show_image(weighted_img(image, raw_filtered_lines_img), None, pipeline_demo_output_dir + '6_raw_filtered_lines.jpg')

lines_img = draw_mean_lines(filtered_lines, image)
show_image(weighted_img(image, lines_img), None, pipeline_demo_output_dir + '7_lines.jpg')

In [ ]:
# Testing pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
test_images_dir = 'test_images/'
test_images_output_dir = 'test_images_output/'

for test_image_filename in os.listdir(test_images_dir): 
    image = mpimg.imread(test_images_dir + test_image_filename)    
    
    res = image_pipeline(image, True)
    show_image(res, None, test_images_output_dir + test_image_filename)

## Test on Videos

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_video(input_path, output_path):
    mean_tracker.reset()
    in_clip = VideoFileClip(input_path)
    out_clip = in_clip.fl_image(image_pipeline) 
    %time out_clip.write_videofile(output_path, audio=False)

In [ ]:
process_video('test_videos/solidWhiteRight.mp4', 'test_videos_output/solidWhiteRight.mp4')

In [ ]:
process_video('test_videos/solidYellowLeft.mp4', 'test_videos_output/solidYellowLeft.mp4')

## Optional Challenge

In [ ]:
process_video('test_videos/challenge.mp4', 'test_videos_output/challenge.mp4')